In [ ]:
from pymtattl import MTADownloader

In [ ]:
mta_downloader = MTADownloader()

In [ ]:
mta_downloader.download_to_txt()

In [ ]:
db_path = mta_downloader.download_to_db()

# Use Staion Name and Google Maps API to get latitude and longitude

In [ ]:
import sqlite3
import os
import pandas as pd

In [ ]:
res = pd.read_sql_query("select * from name_keys", conn)

In [ ]:
res.head()

In [ ]:
nk_append = [('R194','R217','BLEECKER ST','6DF','IRT'),
             ('R001','R101','SOUTH FERRY','R1','IRT'),
             ('R028','A077','FULTON ST','ACJZ2345','BMT'),
             ('R028','A081','FULTON ST','ACJZ2345','BMT'),
             ('R028','A082','FULTON ST','ACJZ2345','BMT'),
             ('R088','A049','CORTLANDT ST','R','BMT'),
             ('R057','R612','ATLANTIC AVE','2345BDNQR','IRT'),
             ('R028','N098','FULTON ST','2345ACJZ','IRT'),
             ('R202','N330','63 DR-REGO PARK','MR','IND'),
             ('R168','R169','96 ST','123','IRT'),
             ('R014','N095A','FULTON ST','ACJZ2345','IND')]

def append_name_key(db_path, new_nk):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.executemany('INSERT INTO name_keys VALUES '
              '(?,?,?,?,?)', new_nk)
    conn.commit()
    conn.close()
    return

In [ ]:
append_name_key(db_path, nk_append)

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
import json

GEOCODING_ENDPOINT = 'https://maps.googleapis.com/maps/api/geocode/json?address='

# metrocard vans could be ignored as no turnstile data and doesn't sound like going to have any
# newark hw bmebe, newark bm bw, newark hm he no idea
miss_station_dict = {
    "2 BDWY CUST SRV":"2 Broadway",
    "8 ST-B'WAY NYU":"East 8th Street & Broadway",
    "HOYT/SCHERMER":"Hoyt Street & Schermerhorn Street",
    "MURRAY ST-B'WAY":"Broadway/Murray St",
    "PRINCE ST-B'WAY":"Broadway/Prince St"
}

def search_geocoding(name):
    full = name.replace(" ","+") + ",+New+York"
    req = GEOCODING_ENDPOINT + full + "&key=" + GEOCODING_API_KEY
    data = json.load(urlopen(req))
    return data['results'][0]['geometry']['location']

def locate_stations(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS geo_stations '
              '(station text, lat real, lng real)')
    stations = pd.read_sql_query("select distinct station from name_keys", conn)['station']
    exist_stations = pd.read_sql_query("select distinct station from geo_stations", conn)['station']
    new_set = set(stations)-set(exist_stations)
    new_stations = {}
    miss_stations = []
    j = 0
    for s in new_set:
        try:
            new_stations[s] = search_geocoding(s)
        except HTTPError as he:
            print(he)
            conn.close()
            raise
        except IndexError as e:
            if s in miss_station_dict.keys():
                new_stations[s] = search_geocoding(miss_station_dict[s])
            else:
                miss_stations.append(s)
                continue
        j+=1
        if j % 20 == 0:
            print("Searching {} stations...".format(j))
    df = pd.DataFrame.from_dict(new_stations, orient='index').reset_index()
    df.rename(index=str, columns={"index":"station"}, inplace=True)
    df.to_sql('geo_stations', con=conn, if_exists='append', index=False)
    conn.close()
    print("Wrote {} out of {} new station locations to database".format(j, len(new_set)))
    return miss_stations

In [ ]:
miss_stations = locate_stations(db_path)

In [ ]:
miss_stations

# Daily Station Summary

In [ ]:
START = "2010-04-17"
END = "2010-04-21"

In [350]:
def daily_station_summary(db_path, start, end, geo=True):
    conn = sqlite3.connect(db_path)
    QUERY_TEXT = "select * from turnstiles where date >= '{}' and date <= '{}' and desc = 'REGULAR'".format(start, end)
    df = pd.read_sql_query(QUERY_TEXT, conn)
    df_nk = pd.read_sql_query("select * from name_keys", conn)
    df_nk.drop(['line','division'], axis=1, inplace=True)
    if geo:
        df_geo = pd.read_sql_query("select * from geo_stations", conn)
    conn.close()
    
    # de-cumulate entry/exit numbers
    df['datime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.drop(['date','time','desc'], axis=1, inplace=True)
    df.drop_duplicates(keep='first', inplace=True)
    level = ['remote','booth','scp']
    df = df.sort_values(by=level + ['datime']).reset_index(drop=True)
    df['entry_diff'] = df.groupby(level)['entries'].diff()
    df['exit_diff'] = df.groupby(level)['exits'].diff()
    # might have negative values due to reasons ie. counter reset, etc.
    # but there shouldn't be negative entries/exits, set to zero
    df.loc[df['entry_diff'] < 0, 'entry_diff'] = 0
    df.loc[df['exit_diff'] < 0, 'exit_diff'] = 0
    df['date'] = df['datime'].dt.date
    df.drop(['scp','entries','exits','datime'], axis=1, inplace=True)
    
    # get station name from name_keys table
    df = df.merge(df_nk, how='inner', left_on=['booth','remote'], right_on=['booth','remote']).fillna(0)
    df = df.groupby(['station','date'])['entry_diff','exit_diff'].sum().reset_index()
    if geo:
        df = df.merge(df_geo, how='inner', on='station')
    return df

In [351]:
test_df = daily_station_summary(db_path, START, END)

In [352]:
test_df.head()

,station,date,entry_diff,exit_diff,lat,lng
0,1 AVE,2010-04-17,12441.0,14374.0,40.763368,-73.95924
1,1 AVE,2010-04-18,15255.0,17258.0,40.763368,-73.95924
2,1 AVE,2010-04-19,20346.0,22223.0,40.763368,-73.95924
3,1 AVE,2010-04-20,21263.0,23791.0,40.763368,-73.95924
4,1 AVE,2010-04-21,21786.0,25174.0,40.763368,-73.95924
